# Azure Digital Twins with Python: A Basic Tutorial

In the recent days, I have been trying to learn about digital twins and how to use them. Hence, I am making this short tutorial as a future reference to myself and perhaps also to others who might be trying to do the same. Of course, being able to create and manipulate them programmatically is part of that endeavor. More specifically, how to work with Azure Digital Twins using Python.

Lets begin by first, defining what a digital twin is. While there are many definitions, I like the one from IBM :

_A virtual model designed to accurately reflect a physical object._

## Creating the Virtual Environment

```sh
azure-common==1.1.28
azure-core==1.26.2
azure-digitaltwins-core==1.2.0
azure-identity==1.12.0
azure-mgmt-authorization==3.0.0
azure-mgmt-core==1.3.2
azure-mgmt-digitaltwins==6.4.0
azure-mgmt-resource==22.0.0
```

The version of the modules that I am working with in this tutorial are included above.

## Create the resource group

To do anything in Azure, we first need to create a resource group. So lets create one called 'Tutorial-RG'.

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from pprint import pprint

### Find out your Subscription ID

In [ ]:
!az account list -o table

In [ ]:
credential = DefaultAzureCredential()
subscription_id = "00000000-0000-0000-0000-000000000000" # use your own

In [ ]:
resource_client = ResourceManagementClient(
    credential, subscription_id=subscription_id)

In [ ]:
resource_group_name = "Tutorial-RG"
rg_result = resource_client.resource_groups.create_or_update(
    resource_group_name, {"location": "westeurope"}
)

In [ ]:
pprint(rg_result)

# Instantiating a digital twin 

In [ ]:
from azure.mgmt.digitaltwins import AzureDigitalTwinsManagementClient

In [ ]:
client = AzureDigitalTwinsManagementClient(
    credential = DefaultAzureCredential(),
    subscription_id = subscription_id,
)

In [ ]:
dt_resource_name = "Tutorial-DT"
dt_response = client.digital_twins.begin_create_or_update(
    resource_group_name=rg_result.name,
    resource_name = dt_resource_name,
    digital_twins_create={"location": "westeurope"},
).result()
print(dt_response)

In [ ]:
print(dt_response.host_name)

### Adding the Azure Digital Twin Data Owner Role

In [ ]:
from azure.mgmt.authorization import AuthorizationManagementClient
import uuid

In [ ]:
# resource_client = ResourceManagementClient(
#     credential=DefaultAzureCredential(),
#     subscription_id=subscription_id
# )

In [ ]:
# https://github.com/Azure-Samples/azure-samples-python-management/blob/main/samples/authorization/manage_role_assignment.py#L1
authorization_client = AuthorizationManagementClient(
    credential=DefaultAzureCredential(),
    subscription_id=subscription_id,
    api_version="2018-01-01-preview"
)

adt_data_owner_role_id ='bcd981a7-7f74-457b-83e1-cceb9e632ffe'
scope = f'/subscriptions/{subscription_id}/resourceGroups/{resource_group_name}'

try:
    role_assignment = authorization_client.role_assignments.create(
        scope,
        str(uuid.uuid4()),
        {
            "role_definition_id": "/subscriptions/" 
            + subscription_id 
            + "/providers/Microsoft.Authorization/roleDefinitions/" 
            + adt_data_owner_role_id,
            "principal_id": 'use-your-principal-id', # get your principal ID from azure portal
        }
    )
    print(role_assignment)
except Exception as e:
    print(e)


In [ ]:
# https://learn.microsoft.com/en-us/azure/digital-twins/concepts-security#authorization-azure-roles-for-azure-digital-twins
# https://learn.microsoft.com/en-us/azure/digital-twins/how-to-authenticate-client#assign-an-access-role
# give yourself a Data Owner Role
# !az dt role-assignment create --dt-name myDigitalTwinsService --assignee "<USE YOUR PRINCIPAL ID>" --role "Azure Digital Twins Data Owner" --debug

# Digital Twin Instantiation via Portal (old)

follow Microsoft's guide to setup your Azure digital twin instance and proper authentication.

https://learn.microsoft.com/en-us/azure/digital-twins/how-to-set-up-instance-portal

## Azure DT Service Client

Lets create service client to access the SDK functions for digital twin manipulation.


In [ ]:
from azure.digitaltwins.core import DigitalTwinsClient

In [ ]:
# Replace the instance url of your digital twin here
adtInstanceUrl = f"https://{dt_response.host_name}"

dt_client = DigitalTwinsClient(adtInstanceUrl, DefaultAzureCredential())
print("Service Client Created! Ready to go")

In [ ]:
adtInstanceUrl


## Uploading a Model

We need a model to base our digital off of. Below, we have a sample model as a `dict`. However, we can also load one from a json file.

Microsoft provides an detailed write up about the concepts of models for digital twins.

https://learn.microsoft.com/en-us/azure/digital-twins/concepts-models

In [ ]:
sample_model = {
    "@id": "dtmi:example:SampleModel;1",
    "@type": "Interface",
    "displayName": "SampleModel",
    "contents": [
        {
            "@type": "Relationship",
            "name": "contains"
        },
        {
            "@type": "Property",
            "name": "data",
            "schema": "string"
        },
        {
            "@type": "Telemetry",
            "name": "sensor",
            "schema": "double"      
        }
    ],
    "@context": "dtmi:dtdl:context;2"
}

In [ ]:
# if you have the model defined as a seperate json, just read it like so:

# with open("SampleModel.json", 'r') as f:
#     sample_model = json.load(f)
# pprint(sample_model)

It's good practice to put our call for model creation inside a try-except statement. This is because we cannot upload two models with the same id and subsequent calls with the models having the same id will throw an error.

In [ ]:
try:
    # put all the models that we want to create into a list
    models = [sample_model]
    dt_client.create_models(models)
    print("Model uploaded to the instance")
except Exception as e:
    print(e)

<!-- ![title](images/model_upload.png) -->
It should show up in the model explorer as follows:

<img src="images/model_upload.png" width="800" height="500" />

## Retreive uploaded models

Now that we have creaated our models, lets make call to retreive them.

In [ ]:
# Get list of models
models_list = dt_client.list_models()
for m in models_list:
    print(m.id)

In [ ]:
# For now, we just have a single model
# We can also retreive a model's information as a dict
m.as_dict()

## Creating a digital twin
Now that we have uploaded a model, we can use it to create a digital twin out of it.

In [ ]:
# unique id of the digital twin
digital_twin_id = 'digiTwin-1'

# the necessary metadata of the digital twin
temp_twin = {
    "$metadata": {
        "$model": "dtmi:example:SampleModel;1" #give the model's id here
    },
    "$dtId": digital_twin_id,
    "data": "Hello World"
}
created_twin = dt_client.upsert_digital_twin(digital_twin_id, temp_twin)
print("DT Created")

Lets create a few more twins. 

Our digital twin should show up in the model explorer as follows:
(Click the run query button with the default "SELECT * FROM digitaltwins")

<img src="images/digital_twin_1.png" width="800" height="500"  />

Lets create 2 more twins. Unlike uploading a model, no error is thrown if a digital twin with the same ID already exists. An attempt to create the same twin again will just replace the original twin.

In [ ]:
# creating some twins
prefix = "digiTwin-"
for i in range(1, 4):
    twin_id = f'{prefix}{i}'
    temp_twin = {
        "$metadata": {
            "$model": "dtmi:example:SampleModel;1"
        },
        "$dtId": twin_id,
        "data": "Hello World"
    }
    created_twin = dt_client.upsert_digital_twin(twin_id, temp_twin)
    print(f"Created twin: {created_twin['$dtId']}")

Azure Digital Twin explorer should show the newly created twin. 
Don't forget to click the `Run Query` button for it to show up.

<img src="images/digital_twin_2.png" width="800" height="500"  />


## Relationships

Next, we will create relationships among the twins that we have created so that we can connect them in a so-called `twin graph`.
Quoting from Microsoft:
```
A digital twin is an instance of one of your custom-defined models. It can be connected to other digital twins via relationships to form a twin graph: this twin graph is the representation of your entire environment.
```

https://learn.microsoft.com/en-us/azure/digital-twins/concepts-twins-graph

Recall that our model has the a relationship called `contains`. In the example below, we use that to create a contains relationship from `digiTwin-1` to `digiTwin-2` and `digiTwin-3`.

In [ ]:
# Creating relationships
for i in [2, 3]:
    relationship = {
        "$relationshipId": f"contains-{i}", # give the relationship some unique id
        "$sourceId": f'{prefix}{1}', # give the id of the source twin
        "$relationshipName": "contains", # give the relationship name as defined in the model
        "$targetId": f'{prefix}{i}', # give the id of the target twin
    }
    # make the call to create the relationship
    rel = dt_client.upsert_relationship(
        relationship["$sourceId"],
        relationship["$relationshipId"],
        relationship
    )
    print(
        f"Created Relationship '{rel['$relationshipName']}' from {rel['$sourceId']} to {rel['$targetId']}")


The Azure DT explorer should also show the relationships:
(May need to wait a minute or two for it to show up sometimes)

<img src="images/relationship_1.png" width="800" height="500"  />


Lets list the created relationships:

In [ ]:
created_relationships = dt_client.list_relationships(f'{prefix}{1}')
for r in created_relationships:
    pprint(r)

In [ ]:
# The relationships can also be viewed from the target twins
for i in range(1, 4):
    incoming_relationships = dt_client.list_incoming_relationships(
        f'{prefix}{i}')
    for r in incoming_relationships:
        pprint(f"{prefix}{i} has incoming relationship: {r.as_dict()}")


## Update Property

Recall that each twin has a property called data. So, far we have let that be just "hello world". We can update the property as follows:

In [ ]:
# update property of dt
digital_twin_id = "digiTwin-1"
data_update = {
    "$metadata": {
        "$model": "dtmi:example:SampleModel;1"
    },
    'data': f'New Data for {digital_twin_id}'
}
dt_client.upsert_digital_twin(digital_twin_id, data_update)

Notice that the data changed in the properties panel for `digiTwin-1`.

<img src="images/data_update.png" width="800" height="500"  />

## Publish Telemetry

In [ ]:
# update property of dt
digital_twin_id = "digiTwin-1"
telemetry_payload = '{"sensor": 10.1}'
dt_client.publish_telemetry(
    digital_twin_id,
    telemetry_payload
)

## Queries

We can retreive the twins created so far in our digital instance by running queries.
Lets just run the `SELECT * FROM digitaltwins` query for now:

https://learn.microsoft.com/en-us/azure/digital-twins/concepts-query-language

In [ ]:
query_expression = 'SELECT * FROM digitaltwins'
query_result = dt_client.query_twins(query_expression)
print('DigitalTwins:')
for twin in query_result:
    print(twin['$dtId'])


Thats about it for this basic tutorial. Lets delete every twin next:

In [ ]:
digital_twin_id = 'digiTwin-1'
dt_client.delete_digital_twin(digital_twin_id)  # this will fail

We can't just delete a twin directly that has active relationships. So we have to delete the relationships first before we can delete the twin:

In [ ]:
# Delete all active relationship (needed to delete twins)
query_expression = 'SELECT * FROM digitaltwins'
query_result = dt_client.query_twins(query_expression)  # get all twins
print('DigitalTwins:')
for twin in query_result:
    # print(twin)
    digital_twin_id = twin['$dtId']
    relationships = dt_client.list_relationships(digital_twin_id) # list the twins relationships
    for relationship in relationships:
        relationship_id = relationship['$relationshipId']
        dt_client.delete_relationship(digital_twin_id, relationship_id) # delete the relationship
        print(f'Deleted: {relationship} from {digital_twin_id}')

In [ ]:
# Now we can delete all the twins
query_expression = 'SELECT * FROM digitaltwins'
query_result = dt_client.query_twins(query_expression)
for twin in query_result:
    digital_twin_id = twin['$dtId']
    dt_client.delete_digital_twin(digital_twin_id)
    print(f'Deleted twin: {digital_twin_id}')

Lets also delete the models:

In [ ]:
for model in dt_client.list_models():
    print(model)
    dt_client.delete_model(model.id)
    print(f'Deleted : {model.id}')

In [ ]:
# delete resource group
poller = resource_client.resource_groups.begin_delete(rg_result.name)
result = poller.result()


END